In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tpot import TPOTRegressor
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
from featuresv2 import features, targets, metadata
import json
import requests
%matplotlib inline

## Version 2 Model

In [17]:
data = pd.read_csv('outputs/model_training_data_v2.csv')
data_2023 = data[data['round.year']!= 2023]

In [3]:
round_matchups = [
    ('Sydney Swans','Melbourne'),
    ('Brisbane Lions','Carlton'),
    ('Gold Coast Suns','Richmond'),
    ('GWS Giants','Collingwood')]

In [4]:
round1 = [('Carlton', 'Richmond'),
('Collingwood','Sydney Swans'),
('Essendon','Hawthorn'),
('GWS Giants','North Melbourne'),
('Geelong Cats','St Kilda'),
('Gold Coast Suns','Adelaide Crows'),
('Melbourne','Western Bulldogs'),
('Port Adelaide','West Coast Eagles'),
('Fremantle','Brisbane Lions')]

In [5]:
targets

['homeTeamScore.matchScore.totalScore',
 'awayTeamScore.matchScore.totalScore',
 'score_diff']

In [3]:
def generate_test_train_split(data, features, target, test_size=0.3):
    X = data[features]
    y = data[target]

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    return {"x_train":x_train,"x_test":x_test,"y_train":y_train,"y_test":y_test}

In [4]:
def tpot_pipeline_v2(data, features, target, size=20, scoring='r2'):
    data_ = generate_test_train_split(data, features, target)

    my_scorer = make_scorer(mean_squared_error, greater_is_better=False)

    pipeline_optimizer = TPOTRegressor(generations=5, population_size=size, verbosity=2, scoring=scoring)

    pipeline_optimizer.fit(data_['x_train'], data_['y_train'])
    
    print(pipeline_optimizer.score(data_['x_test'], data_['y_test']))
    
    return pipeline_optimizer

In [5]:
def return_stats_input(data, home_team, away_team, model_inputs, games=5):
    '''Take in player_stats and returns aggregated team stats for previous home/away team matchups'''
    
#     if 'team.name' not in model_inputs:
#         model_inputs.append('team.name')
        
    match_ups = data[(data['match.homeTeam.name']==home_team) & (data['match.awayTeam.name']==away_team)]
    weights = range(1,len(match_ups)+1)

    stats = match_ups[model_inputs+['match.homeTeam.name']].groupby('match.homeTeam.name').apply(lambda x: np.average(x, weights=weights, axis=0))
    
    return pd.DataFrame(dict(zip(model_inputs,stats[0])),index=[0])

In [6]:
def generate_margin_tear_off(model, round_matchups, team_stats, model_inputs):
    print('-------------------' + str(model.fitted_pipeline_[0]).replace('()','') + ' TEAR-OFF-------------------')
    for tup in round_matchups:
        predict = model.predict(return_stats_input(team_stats, tup[0], tup[1], model_inputs))[0]
        if predict < 0:
            winner = tup[1]
        else:
            winner = tup[0]
        print("------------------------------------------------")
        print("Home: " + tup[0])
        print("Away: " + tup[1])
        print("Winner: " + winner + "\t       " + "Margin: " + str(abs(round(predict))))

In [23]:
def generate_score_tear_off(home_model, away_model, round_matchups, team_stats, model_inputs):
   # print('-------------------' + str(model.fitted_pipeline_[0]).replace('()','') + ' TEAR-OFF-------------------')
    preds = []
    for tup in round_matchups:
        try:
            h_predict = home_model.predict(return_stats_input(team_stats, tup[0], tup[1], model_inputs))[0]
            a_predict = away_model.predict(return_stats_input(team_stats, tup[0], tup[1], model_inputs))[0]
            if h_predict < a_predict:
                winner = tup[1]
                margin = round(a_predict) - round(h_predict)
            else:
                winner = tup[0]
                margin = round(h_predict) - round(a_predict)
            paddingh = 25 - len("Home: " + tup[0])
            paddinga = 25 - len("Away: " + tup[1])
            print("------------------------------------------------")
            print("Home: " + tup[0] + " "*paddingh + str(abs(round(h_predict))))
            print("Away: " + tup[1] + " "*paddinga + str(abs(round(a_predict))))
            print("Winner: " + winner + " by "+ str(margin) + " points")
            print("Total Score: " + str(round(h_predict) + round(a_predict)))
            preds.append((h_predict, a_predict, margin))
        except:
            print("No prior game")
            preds.append((0, 0, 0))
    return preds

In [19]:
home_model = tpot_pipeline_v2(data, features, targets[0], size=10)

Optimization Progress:   0%|          | 0/60 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9618313180687397

Generation 2 - Current best internal CV score: 0.9618313180687397

Generation 3 - Current best internal CV score: 0.967755768100042

Generation 4 - Current best internal CV score: 0.9688824525915326

Generation 5 - Current best internal CV score: 0.9715880029036713

Best pipeline: LinearSVR(StandardScaler(input_matrix), C=25.0, dual=False, epsilon=0.001, loss=squared_epsilon_insensitive, tol=1e-05)
0.9732087873267681


In [20]:
away_model = tpot_pipeline_v2(data, features, targets[1], size=10)

Optimization Progress:   0%|          | 0/60 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.973307803532746

Generation 2 - Current best internal CV score: 0.973307803532746

Generation 3 - Current best internal CV score: 0.973307803532746

Generation 4 - Current best internal CV score: 0.973307803532746

Generation 5 - Current best internal CV score: 0.973307803532746

Best pipeline: RidgeCV(SelectPercentile(input_matrix, percentile=66))
0.9724739319520406


In [ ]:
margin_model = tpot_pipeline_v2(data, features, targets[2], size=10)

In [ ]:
generate_score_tear_off(home_model, away_model, round1, data, features)

In [ ]:
# print(return_stats_inputv3(data, home_team, away_team, features))
generate_margin_tear_off(margin_model, round1, data, features)
# data[(data['match.homeTeam.name']==home_team) & (data['match.awayTeam.name']==away_team)][features].head()

In [24]:
fix_data_2024 = json.loads(requests.get('https://fixturedownload.com/feed/json/afl-2024').content)

fix_data_2023 = json.loads(requests.get('https://fixturedownload.com/feed/json/afl-2023').content)

fixtures2024 = pd.DataFrame(fix_data_2024)
games2024 = list(zip(fixtures2024.HomeTeam, fixtures2024.AwayTeam))

fixtures2023 = pd.DataFrame(fix_data_2023)
games2023 = list(zip(fixtures2023.HomeTeam, fixtures2023.AwayTeam))

predicted_season = generate_score_tear_off(home_model, away_model, games2023, data_2023, features)

fixtures2023['predicted_home'] = [round(i[0]) for i in predicted_season]
fixtures2023['predicted_away'] = [round(i[1]) for i in predicted_season]

------------------------------------------------
Home: Richmond           92
Away: Carlton            70
Winner: Richmond by 22 points
Total Score: 162
------------------------------------------------
Home: Geelong Cats       77
Away: Collingwood        52
Winner: Geelong Cats by 25 points
Total Score: 129
------------------------------------------------
Home: North Melbourne    59
Away: West Coast Eagles  56
Winner: North Melbourne by 3 points
Total Score: 115
------------------------------------------------
Home: Port Adelaide      52
Away: Brisbane Lions     99
Winner: Brisbane Lions by 47 points
Total Score: 151
------------------------------------------------
Home: Melbourne          102
Away: Western Bulldogs   74
Winner: Melbourne by 28 points
Total Score: 176
------------------------------------------------
Home: Gold Coast Suns    100
Away: Sydney Swans       56
Winner: Gold Coast Suns by 44 points
Total Score: 156
No prior game
------------------------------------------------

------------------------------------------------
Home: Carlton            84
Away: Brisbane Lions     90
Winner: Brisbane Lions by 6 points
Total Score: 174
------------------------------------------------
Home: Richmond           108
Away: West Coast Eagles  82
Winner: Richmond by 26 points
Total Score: 190
------------------------------------------------
Home: Geelong Cats       94
Away: Adelaide Crows     61
Winner: Geelong Cats by 33 points
Total Score: 155
------------------------------------------------
Home: Gold Coast Suns    54
Away: Melbourne          94
Winner: Melbourne by 40 points
Total Score: 148
------------------------------------------------
Home: GWS Giants         88
Away: Western Bulldogs   106
Winner: Western Bulldogs by 18 points
Total Score: 194
------------------------------------------------
Home: Fremantle          92
Away: Hawthorn           75
Winner: Fremantle by 17 points
Total Score: 167
------------------------------------------------
Home: Port Adelaid

------------------------------------------------
Home: Sydney Swans       116
Away: West Coast Eagles  41
Winner: Sydney Swans by 75 points
Total Score: 157
------------------------------------------------
Home: Fremantle          48
Away: Essendon           87
Winner: Essendon by 39 points
Total Score: 135
------------------------------------------------
Home: Collingwood        106
Away: Adelaide Crows     54
Winner: Collingwood by 52 points
Total Score: 160
------------------------------------------------
Home: Gold Coast Suns    116
Away: Hawthorn           66
Winner: Gold Coast Suns by 50 points
Total Score: 182
------------------------------------------------
Home: Brisbane Lions     93
Away: Richmond           89
Winner: Brisbane Lions by 4 points
Total Score: 182
------------------------------------------------
Home: Sydney Swans       89
Away: Geelong Cats       86
Winner: Sydney Swans by 3 points
Total Score: 175
------------------------------------------------
Home: Western 

------------------------------------------------
Home: North Melbourne    70
Away: Essendon           106
Winner: Essendon by 36 points
Total Score: 176
------------------------------------------------
Home: Sydney Swans       85
Away: Gold Coast Suns    68
Winner: Sydney Swans by 17 points
Total Score: 153
------------------------------------------------
Home: Brisbane Lions     97
Away: Adelaide Crows     58
Winner: Brisbane Lions by 39 points
Total Score: 155
------------------------------------------------
Home: Carlton            70
Away: Melbourne          72
Winner: Melbourne by 2 points
Total Score: 142
------------------------------------------------
Home: West Coast Eagles  79
Away: Fremantle          87
Winner: Fremantle by 8 points
Total Score: 166
------------------------------------------------
Home: Hawthorn           62
Away: Western Bulldogs   72
Winner: Western Bulldogs by 10 points
Total Score: 134
------------------------------------------------
Home: St Kilda      

In [25]:
pred_winner = []
winner = []
for i, row in fixtures2023.iterrows():

    if row[7] > row[8]:
        winner.append(row[4])
    elif row[8] > row[7]:
        winner.append(row[5])
    else:
        winner.append('Draw')


    if row[9] > row[10]:
        pred_winner.append(row[4])
    elif row[10] > row[9]:
        pred_winner.append(row[5])
    else:
        pred_winner.append('Draw')

fixtures2023['predicted_winner'] = pred_winner
fixtures2023['winner'] = winner
fixtures2023['predicted_margin'] = fixtures2023['predicted_home'] - fixtures2023['predicted_away']
fixtures2023['actual_margin'] = fixtures2023['HomeTeamScore'] - fixtures2023['AwayTeamScore']
fixtures2023['correct'] = [1 if row[11]==row[12] else 0 for i,row in fixtures2023.iterrows()]
fixtures2023['bet'] = ["1-39" if abs(row[13]) < 40 else "40+" for i,row in fixtures2023.iterrows()]


fixtures2023.to_csv('outputs/predictions_and_results_2023.csv', index=False)

backtest_data = pd.read_csv('outputs/predictions_and_results_2023.csv')
backtest_data.head()

,MatchNumber,RoundNumber,DateUtc,Location,HomeTeam,AwayTeam,Group,HomeTeamScore,AwayTeamScore,predicted_home,predicted_away,predicted_winner,winner,predicted_margin,actual_margin,correct,bet
0,1,1,2023-03-16 08:20:00Z,MCG,Richmond,Carlton,NaN,58,58,92,70,Richmond,Draw,22,0,0,1-39
1,2,1,2023-03-17 08:40:00Z,MCG,Geelong Cats,Collingwood,NaN,103,125,77,52,Geelong Cats,Collingwood,25,-22,0,1-39
2,3,1,2023-03-18 02:45:00Z,Marvel Stadium,North Melbourne,West Coast Eagles,NaN,87,82,59,56,North Melbourne,North Melbourne,3,5,1,1-39
3,4,1,2023-03-18 05:35:00Z,Adelaide Oval,Port Adelaide,Brisbane Lions,NaN,126,72,52,99,Brisbane Lions,Port Adelaide,-47,54,0,40+
4,5,1,2023-03-18 08:25:00Z,MCG,Melbourne,Western Bulldogs,NaN,115,65,102,74,Melbourne,Melbourne,28,50,1,1-39
